In [3]:
import pandas as pd
import numpy as np
from scipy.stats import f_oneway

In [4]:
df = pd.read_json('P2_player_state.json')

In [85]:
df = df.loc[((df['Season']== '21/22'))]
df = df.loc[(df.Competition =='Bundesliga')|(df.empetition =='Ligue 1')|(df.Competition =='Serie A')|(df.Competition =='Premier League')|(df.Competition =='LaLiga')]
cols_with_dash = df.columns[df.isin(['-']).any()]
df[cols_with_dash] = df[cols_with_dash].replace('-', 0, regex=True)

In [86]:
cols_to_convert = [ 'Apperance']
df[cols_to_convert] = df[cols_to_convert].fillna(0)
df[cols_to_convert] = df[cols_to_convert].astype(int)

In [87]:
df = df.loc[:,['Player_id','Apperance']]
df['Active']=0
df.loc[df['Apperance']>=8, 'Active'] = 1
df = df.rename(columns={'Player_id': 'id'})

In [88]:
df_age=pd.read_json('table_for_statistics.json')
df_age=df_age.loc[:,['id','age']]
df_merged =pd.merge(df,df_age,on='id',how='left')
df_merged.age.fillna(df_merged.age.mean(),inplace=True)
df_merged.age = df_merged.age-1

In [89]:
df_temp = pd.read_json('teams.json')
df_temp = df_temp[['id', 'players_ids','year']]
df_temp = df_temp[df_temp['year'] == '2021/2022']
df_temp = df_temp[['id', 'players_ids']]
df_temp = df_temp.rename(columns={'id': 'team_id','players_ids': 'id'})
df_temp = df_temp.reset_index(drop=True)
df_temp = df_temp.explode('id')
df_temp = df_temp.astype(int)

In [90]:
df_final = pd.merge(df_merged,df_temp,on='id',how='left')

In [91]:
bins = df_final.groupby('team_id').agg({'Active':'sum','age': 'mean'})
bins.age.describe()

count    98.000000
mean     25.282438
std       1.052700
min      23.307692
25%      24.565315
50%      25.103134
75%      25.915974
max      27.836036
Name: age, dtype: float64

In [92]:
bins['age_level'] = 'Normal'
bins.loc[bins['age']<=24.5, 'age_level'] = 'Youngest'
bins.loc[bins['age']>=26, 'age_level'] = 'Oldest'
bins.age_level.value_counts()

Normal      55
Youngest    22
Oldest      21
Name: age_level, dtype: int64

In [93]:
g1 = bins[bins.age_level=='Youngest']['Active'].values
g2 = bins[bins.age_level=='Normal']['Active'].values
g3 = bins[bins.age_level=='Oldest']['Active'].values

In [94]:
#ANNOVA
stat, p = f_oneway(g1, g2, g3)

if p < 0.05:
    print("The groups are not from the same distribution.")
else:
    print("The groups are from the same distribution.")

The groups are from the same distribution.
